In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode


def isName(word):
    # 1. The first character is not a digit (other characters can be digits)
    if re.match("[0-9]\w+", word):
        return False

    # 2. The first character is uppercase
    if not re.match("^[A-Z]", word):
        return False

    #  3. All the other characters that are letters are lowercase.
    if len(word) > 1 and re.match("[A-Z]\w+", word[1:]):
        return False
    
    return True


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:myGroup,Total_words,%d" % 1
        if isName(word):
            print "%s\t%d" % (word, 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue

    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer.py


In [18]:
%%writefile myScript.py

import sys
reload(sys)

total_words_file = 'total_words.txt'
with open(total_words_file, 'r') as f:
    for line in f:
        total_words = float(line.strip())

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    
    if float(count) / total_words <= 0.005:
        print "%s\t%d" % (key, count)

Overwriting myScript.py


In [ ]:
%%bash

OUT_DIR="wordcount_result_"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log

# Print result
cat output.log | egrep "*_words" | grep -Eo [0-9]+ > total_words.txt
hadoop fs -cat ${OUT_DIR}/part-* | python2 myScript.py | sort -k2,2nr | head

# print log to stderr for grader
cat output.log >&2